<a href="https://colab.research.google.com/github/prikmm/MelDetect/blob/main/Cluster_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q efficientnet >> /dev/null
!pip install tensorflow-addons tensorflow-hub

     |████████████████████████████████| 706kB 11.0MB/s 


In [3]:
import sys
sys.path.insert(0, '/content/drive/My Drive/Meldetect_libraries')

In [1]:
import pandas as pd, numpy as np
import tensorflow as tf, re, math
import tensorflow.keras.backend as K
import efficientnet.tfkeras as efn
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import os
import cv2
import config
config.REPLICAS, config.AUTO, config.strategy = config.tpu_gpu_initializer(DEVICE="GPU")

config.DATA_PATH = ['gs://kds-e587a221f0797cf8f017657b99d05aee1f262e04db020e07144fa268',
                    'gs://kds-e587a221f0797cf8f017657b99d05aee1f262e04db020e07144fa268',
                    'gs://kds-e587a221f0797cf8f017657b99d05aee1f262e04db020e07144fa268']


config.DATA_PATH2 = ['gs://kds-daa129e989ec157e205de7addb63ac52a6bb066eb3efea7e285641dd',
                    'gs://kds-daa129e989ec157e205de7addb63ac52a6bb066eb3efea7e285641dd',
                    'gs://kds-daa129e989ec157e205de7addb63ac52a6bb066eb3efea7e285641dd']

import models
import datasets
import images
import callbacks
import train

Could not connect to TPU
Using default strategy for CPU and single GPU
Num GPUs Available:  1
REPLICAS: 1


In [10]:
_, featext_model = models.build_model("efficientnet_b6", return_feature_model=True)
featext_model.load_weights("model/weights/efficientnet_b6/feat_efficientnet_b6.h5")

In [2]:
SEED = 42
INC2018 = [0, 0, 0]
INC2019 = [1, 1, 1]
FOLDS = 3
VERBOSE = 2
data_index = None
data_train = None
data_valid = None
data_test = None
IMG_SIZES = [384,384,384]
BATCH_SIZES = [32]*FOLDS


skf = KFold(n_splits=FOLDS,shuffle=True,random_state=SEED)

for fold,(idxT,idxV) in enumerate(skf.split(np.arange(15))):
    if fold != 2:
        continue
    data_index = fold
    data_train = idxT
    data_valid = idxV

files_train = tf.io.gfile.glob([config.DATA_PATH[fold] + '/train%.2i*.tfrec'%x for x in data_train])
if INC2019[fold]:
    files_train += tf.io.gfile.glob([config.DATA_PATH2[fold] + '/train%.2i*.tfrec'%x for x in data_train*2+1])
    print('#### Using 2019 external data')
if INC2018[fold]:
    files_train += tf.io.gfile.glob([config.config.DATA_PATH2[fold] + '/train%.2i*.tfrec'%x for x in data_train*2])
    print('#### Using 2018+2017 external data')
np.random.shuffle(files_train)
files_valid = tf.io.gfile.glob([config.DATA_PATH[fold] + '/train%.2i*.tfrec'%x for x in data_valid])
files_test = np.sort(np.array(tf.io.gfile.glob(config.DATA_PATH[fold] + '/test*.tfrec')))

train_dataset = datasets.get_dataset(files_train, augment=True, shuffle=False, repeat=False,
                            dim=IMG_SIZES[data_index],batch_size = BATCH_SIZES[data_index])

#kmeans_log_path = os.path.join(os.curdir, "kmeans_log")
#os.makedirs(kmeans_log_path, exist_ok=True)
#num_clusters = 2
#kmeans = tf.compat.v1.estimator.experimental.KMeans(
#    num_clusters=num_clusters, model_dir=kmeans_log_path, use_mini_batch=False)

#### Using 2019 external data


## Visualizing Image Embeddings using Tensorboard:

In [5]:
from tensorboard.plugins import projector
import pickle

In [11]:
STEPS = 500
IMG_DIMS = 100

In [67]:
feature_vecs = featext_model.predict(train_dataset)#, steps=STEPS)
pickle.dump(part_feature_vecs, open('model/part_feature_vecs.pkl', 'wb'))

In [12]:
images = None
for item in train_dataset.take(1):
    x, y = item
    images = tf.image.resize(x["img_input"], [IMG_DIMS, IMG_DIMS])
    targets = y
    
# For Visualizing Image Quality
plt.imshow(images[0])

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run ResizeBilinear: Dst tensor is not initialized. [Op:ResizeBilinear]

In [ ]:
# Appending rest of the images to the images tensor
for batch_no, item in enumerate(train_dataset.take(STEPS)):
    if batch_no == 0:
        continue
    x, y = item
    resized_images = tf.image.resize(x["img_input"], [IMG_DIMS, IMG_DIMS])
    images = tf.concat([images, resized_images], 0)
    targets = tf.concat([targets, y])

In [ ]:
print(images.shape)
print(targets.shape)

In [ ]:
prin

In [70]:
PATH = os.getcwd()
LOG_DIR = PATH + '/embedding-tensorboard/log-1'
metadata = os.path.join(LOG_DIR, 'metadata.tsv')
os.makedirs(LOG_DIR, exist_ok=True)

with open(metadata, 'w') as metadata_file:
    for item in train_dataset.take(STEPS):
        x, y = item
        for i in range(x["metadata_input"].shape[0]):
            metadata_file.write('{}\n'.format(tf.cast(y[i], tf.int8)))

In [71]:
def images_to_sprite(data):
    """Creates the sprite image along with any necessary padding
    Args:
      data: NxHxW[x3] tensor containing the images.
    Returns:
      data: Properly shaped HxWx3 image with any necessary padding.
    """
    if len(data.shape) == 3:
        data = np.tile(data[...,np.newaxis], (1,1,1,3))
    data = images_dum = np.array(images)
    data = data.astype(np.float32)
    min = np.min(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) - min).transpose(3,0,1,2)
    max = np.max(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) / max).transpose(3,0,1,2)
    # Inverting the colors seems to look better for MNIST
    #data = 1 - data

    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, 0),
            (0, 0)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant',
            constant_values=0)
    # Tile the individual thumbnails into an image.
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3)
            + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    data = (data * 255).astype(np.uint8)
    return data

In [72]:
sprite = images_to_sprite(images)

In [73]:
cv2.imwrite(os.path.join(LOG_DIR, 'sprite.png'), sprite)

True

In [74]:
with open("model/part_feature_vecs.pkl","rb") as reader:
    part_feature_vecs = pickle.load(reader)

In [75]:
features = tf.Variable(part_feature_vecs, name='feature')

In [76]:
features.shape

TensorShape([21789, 1000])

In [77]:
checkpoint = tf.train.Checkpoint(embedding=features)
checkpoint.save(os.path.join(LOG_DIR, "embedding.ckpt"))

'/home/priyam145/MelDetect/embedding-tensorboard/log-1/embedding.ckpt-1'

In [78]:
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
    # Link this tensor to its metadata file (e.g. labels).
embedding.metadata_path =  'metadata.tsv'
    # Comment out if you don't want sprites
embedding.sprite.image_path =  'sprite.png'
embedding.sprite.single_image_dim.extend([images.shape[1], images.shape[1]])
    # Saves a config file that TensorBoard will read during startup.

projector.visualize_embeddings(LOG_DIR, config)

In [ ]:
valid_dataset = datasets.get_dataset(files_valid, augment=True, shuffle=False, repeat=False,
                            dim=IMG_SIZES[data_index],batch_size = BATCH_SIZES[data_index])

In [ ]:
# train
num_iterations = 10
previous_centers = None
for _ in range(num_iterations):
    kmeans_input = featext_model.predict(train_dataset, steps=5)
    print(kmeans_input)
    print(kmeans_input.shape)
    kmeans.train(kmeans_input)
    cluster_centers = kmeans.cluster_centers()
    if previous_centers is not None:
        print('delta:', cluster_centers - previous_centers)
previous_centers = cluster_centers
print('score:', kmeans.score(input_fn))
print('cluster centers:', cluster_centers)

preds = kmeans.predict_index(test_dataset)